# Image Processing Functions
##### Written by Alperen Arslan

### Necessary Imports

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import array, int32, pad

### PGM Operations

##### PGM Write

In [15]:
# Description:
#
# Write given matrix to PGM file
#
# @param[1]: Image matrix. It must be consist of pixel values.
# @param[2]: Filename. It is name of PGM file that wants to be create.
# @param[3]: Width. Width value of the image.
# @param[4]: Height. Height value of the image.
# @param[5]: Magic Number. Magic number is identifies the file type. PGM file type is P5. Default define is P5.
# @param[6]: Resolution. The value of maximum pixel value. Default define is 255.
#
# @return: No return
def pgmwrite(img, filename, width, height, magicNum='P5', resolution=255):
    # Transform given image array to list
    img = int32(img).tolist()
    
    # Create pmg file
    f = open(filename,'w')
    
    # Header struct
    # P5\n (or P2 / Magic Number)
    # 512 512\n (Width and height value, blank space between these number)
    # 256\n (Resolution value)
    # Data (Binary format)
        
    # Write header to pmg file
    f.write(magicNum + '\n')
    f.write(str(width) + ' ' + str(height) + '\n')
    f.write(str(resolution) + '\n')
    
    # Write data to pmg file
    for i in range(height):
        for j in range(width):
            f.write(chr(img[i][j]))
    f.close()

##### PGM Read

In [16]:
# Description:
#
# Read given matrix to PGM file
#
# @param[1]: Filename. It is name of PGM file that wants to read.
#
# @return[1]: Image Data. The values of image pixels.
# @return[2]: Magic Number. Magic number is identifies the file type. PGM file type is P5. Default define is P5.
# @return[3]: Width. Width value of the image.
# @return[4]: Height. Height value of the image.
# @return[5]: Resolution. The value of maximum pixel value.
#
def pgmread(filename):
    # Open pgm file with binary reading
    f = open(filename,'rb')
    
    # Read all the pgm file
    imageBinary = f.read()

    # Header struct
    # P5\n (or P2 / Magic Number)
    # 512 512\n (Width and height value, blank space between these number)
    # 256\n (Resolution value)
    # Data (Binary format)
    
    # Define variables for header reader while loop
    lineCounter = 0
    cursorIndex = 0
    buffer = ''     # Reading buffer
    magicNum = ''   # Magic number info
    width = ''      # Height info
    height = ''     # Width info
    resolution = '' # Resolution info
    
    # Reading the header struct of pgm file
    # Skip all comments
    # Just read magic number, width, height and resolution
    # Boundry for line number, because of header struct consist of three lines
    while lineCounter < 3:
        # Read binary till encounter the line feed
        # It means, it read one line
        if (chr(imageBinary[cursorIndex]) == '\n'):
            if(lineCounter == 2): # Resolution info
                resolution = buffer
                lineCounter = lineCounter + 1 # Increase line counter
                buffer = '' # Clean buffer
            if (lineCounter == 1): # Width and height info
                [width,height] = buffer.split(' ') # Split buffer to take width and height infos
                lineCounter = lineCounter + 1
                buffer = ''
            if (lineCounter == 0): # Magic num info
                magicNum = buffer
                lineCounter = lineCounter + 1
                buffer = ''
            cursorIndex = cursorIndex + 1
        # Comment skipper block
        if (chr(imageBinary[cursorIndex]) == '#'):
            comment_check = True
            # Skip bytes till encounter line feed and there is no more comment
            while comment_check:
                if (chr(imageBinary[cursorIndex]) == '\n') and (chr(imageBinary[cursorIndex+1]) != '#'):comment_check = False
                cursorIndex = cursorIndex + 1
            buffer = ''
        buffer = buffer + chr(imageBinary[cursorIndex]) # Add bytes to buffer
        cursorIndex = cursorIndex + 1 # Increase cursor index to take next byte to buffer

    # Add rest of the pgm bytes to data vector
    imageData = imageBinary[cursorIndex-1:]
    
    # Return the result
    return (imageData,magicNum,int(width),int(height),int(resolution))

### Transformations

##### Negative of Image

In [18]:
# Description:
#
# The function that taking the negative values of the given image pixels by substracting from resolution
# General form of negative of the image is s = L - 1 -r which is intesity levels in the range [0,L-1]. r is a pixel value of the image.
#
# @param[1]: Image. Image matrix that wants to take negative.
# @param[2]: Height. Height value of the image.
# @param[3]: Width. Width value of the image.
# @param[4]: Resolution. The value of maximum pixel value. Default define is 255.
#
# @return: Matrix. The matrix consist of negative values of the given image.
def imageNegative(image,height,width,resolution=255):
    # Create blank matrix for negative values of the given image
    negative = np.zeros((height,width))

    # for loop that subtracting the every pixel values of given image from resolution
    for heightIndex in range(height):
        for widthIndex in range(width):
            negative[heightIndex][widthIndex] = resolution - image[heightIndex][widthIndex]
    return negative # Return the matrix

##### Log Transformation

In [19]:
# Description:
#
# The function that taking the log transform values of the given image pixels
# General form of the log transformation is s = c*log(1+r). c is a constant. r (pixel value) must be equal or bigger than zero.
#
# @param[1]: Image. Image matrix that wants to take log transform.
# @param[2]: Height. Height value of the image.
# @param[3]: Width. Width value of the image.
# @param[4]: c. Constant value in the log transform formula. Default define is 1.
# @param[5]: Resolution. The value of maximum pixel value. Default define is 255.
#
# @return: Matrix. The matrix consist of log transform values of the given image.
def imageLogTransform(image,height,width,c=1,resolution=255):
    # Create blank matrix for log transformations values of the given image
    imageLogTransform = np.zeros((height,width))

    # for loop that taking the log transform of every pixel values
    for heightIndex in range(height):
        for widthIndex in range(width):
            imageLogTransform[heightIndex][widthIndex] = c * np.log10(1 + image[heightIndex][widthIndex])
    return imageLogTransform # Return the matrix

##### Power Law (Gamma) Transformation

In [20]:
# Description:
#
# The function that taking the gamma transform values of the given image pixels
# General form of the gamma transformation is s = c*(r^gamma). c is a constant. r (pixel value). omega is variable.
#
# @param[1]: Image. Image matrix that wants to take gamma transform.
# @param[2]: Omega. Omega value in the omega transformation formula.
# @param[3]: Height. Height value of the image.
# @param[4]: Width. Width value of the image.
# @param[5]: c. Constant value in the log transform formula. Default define is 1.
# @param[6]: Resolution. The value of maximum pixel value. Default define is 255.
#
# @return: Matrix. The matrix consist of gamma transform values of the given image.
def gammaTransformation(image,omega,height,width,c=1,resolution=255):
    # Create blank matrix for gamma transformations values of the given image
    imageGammaTransform = np.zeros((height,width))

    # for loop that taking the gamma transform of every pixel values
    for heightIndex in range(height):
        for widthIndex in range(width):
            imageGammaTransform[heightIndex][widthIndex] = c * ((image[heightIndex][widthIndex])**(omega))
    return imageGammaTransform # Return the matrix

### Filtering


##### Lowpass Filter Creator

In [17]:
# Description:
#
# Create lowpass filter which is a matrix consist of ones
#
# @param[1]: Height. Height of lowpass filters matrix wants to be create. Must be odd number.
# @param[2]: Width. Width of lowpass filters matrix wants to be create. Must be odd number.
#
# @return: Lowpass filter matrix. Lowpass filter matrix wants to be create.
def lowpassFilter(height,width):
    # Check height and width inputs are odd or not
    if (height % 2 == 0) and (width % 2 == 0):
        error = 'Filter dimesions must be odd number!'
        return error # Return the error message
    # Create the matrix if there is no error
    lowpass = np.ones((height,width))
    return lowpass # Return the matrix

##### Filter Applicator

In [ ]:
# Description:
#
# The function that apply the given filter to given one pixel for using in other functions
#
# @param[1]: Image. Image matrix that wants to apply filter
# @param[2]: Filter. Filter that wants to apply
# @param[3]: Pad number. Number of added pad frame to image matrix
# @param[4]: Height index. Height value of the pixel that wants to apply filter
# @param[5]: Width index. Width value of the pixel that wants to apply filter
#
# @return: Pixel value. Pixel value that applied filter.
def filterApplicator(image,filter,padNumber,heightIndex,widthIndex):
    # Initial definition of index values for for loop
    pixelValue = 0
    imageHeightIndex = heightIndex-padNumber # Height start index of image matrix
    imageWidthIndex = widthIndex-padNumber # Width start index of image matrix
    
    # for loop that apply filter to given pixel
    for filterHeightIndex in range(len(filter)):
        for filterWidthIndex in range(len(filter)):
            pixelValue = pixelValue + (image[imageHeightIndex][imageWidthIndex]*filter[filterHeightIndex][filterWidthIndex])
            imageWidthIndex = imageWidthIndex + 1
        imageWidthIndex = widthIndex-padNumber # Reset the width start index of the image matrix
        imageHeightIndex = imageHeightIndex + 1
    
    # Variable definition for normalizing the pixel value
    filterValueSum = 0
    
    # Summing the filter values for pixel normalization
    for i in range(len(filter)):
        for j in range(len(filter)):
            filterValueSum = filterValueSum + filter[i][j]
    pixelValue = pixelValue / filterValueSum
    
    return int(pixelValue) # Return the pixel value

##### Image Smoother

In [ ]:
# Description:
#
# The function that smooth the given image by given filter
#
# @param[1]: Image. Image matrix that wants to smooth
# @param[2]: Filter. Filter that wants to apply
# @param[3]: Pad number. Number of added pad frame to image matrix
# @param[4]: Height index. Height value of the pixel that wants to apply filter
# @param[5]: Width index. Width value of the pixel that wants to apply filter
#
# @return: Matrix. The matrix that consist of smoothed pixel values of the given image
def imageSmoother(matrix,filter,padNumber,height,width):
    # Create blank matrix for image smoothing
    smoothedImage = np.zeros((height,width))

    # for loop that smooth the given image
    for heightIndex in range(height-1):
        for widthIndex in range(width-1):
            smoothedImage[heightIndex][widthIndex] = filterApplicator(matrix,filter,padNumber,heightIndex,widthIndex)
    return smoothedImage # Return the matrix